## `using` 声明和显式资源管理TypeScript 5.2 添加了对 ECMAScript 中即将推出的[显式资源管理](https://github.com/tc39/proposal-explicit-resource-management)功能的支持。让我们探讨一下这个功能的一些动机，并了解它为我们带来了什么。在创建对象后，通常需要进行某种“清理”工作。例如，您可能需要关闭网络连接、删除临时文件，或者仅仅是释放一些内存。让我们想象一个函数，它创建一个临时文件，对其进行各种操作的读写，然后关闭并删除它。

In [ ]:
import * as fs from "fs";export function doSomeWork() {    const path = ".some_temp_file";    const file = fs.openSync(path, "w+");    // use file...    // Close the file and delete it.    fs.closeSync(file);    fs.unlinkSync(path);}

这很好，但如果我们需要在执行过程中提前退出，会发生什么？

In [ ]:
export function doSomeWork() {    const path = ".some_temp_file";    const file = fs.openSync(path, "w+");    // use file...    if (someCondition()) {        // do some more work...        // Close the file and delete it.        fs.closeSync(file);        fs.unlinkSync(path);        return;    }    // Close the file and delete it.    fs.closeSync(file);    fs.unlinkSync(path);}

我们开始看到一些重复的清理工作，这很容易被忘记。如果抛出错误，我们也不能保证关闭并删除文件。通过将所有内容包装在 `try`/`finally` 块中可以解决这个问题。

In [ ]:
export function doSomeWork() {    const path = ".some_temp_file";    const file = fs.openSync(path, "w+");    try {        // use file...        if (someCondition()) {            // do some more work...            return;        }    }    finally {        // Close the file and delete it.        fs.closeSync(file);        fs.unlinkSync(path);    }}

虽然这样做更加健壮，但它给我们的代码增加了很多“噪音”。如果我们在`finally`块中添加更多的清理逻辑，我们还会遇到其他潜在的问题，例如，异常阻止其他资源被释放。这就是[显式资源管理](https://github.com/tc39/proposal-explicit-resource-management)提案旨在解决的问题。该提案的关键思想是支持资源处置——我们试图处理的这种清理工作——作为JavaScript中的一等概念。首先，它添加了一个名为`Symbol.dispose`的新内置`symbol`，我们可以创建具有以`Symbol.dispose`命名的方法的对象。为了方便起见，TypeScript定义了一个名为`Disposable`的新全局类型，用于描述这些对象。

In [ ]:
class TempFile implements Disposable {    #path: string;    #handle: number;    constructor(path: string) {        this.#path = path;        this.#handle = fs.openSync(path, "w+");    }    // other methods    [Symbol.dispose]() {        // Close the file and delete it.        fs.closeSync(this.#handle);        fs.unlinkSync(this.#path);    }}

稍后我们可以调用这些方法。

In [ ]:
export function doSomeWork() {    const file = new TempFile(".some_temp_file");    try {        // ...    }    finally {        file[Symbol.dispose]();    }}

将清理逻辑移动到`TempFile`本身并没有给我们带来太多好处；我们基本上只是将所有的清理工作从`finally`块中移动到一个方法中，而这一直是可能的。但是，有一个众所周知的“名称”意味着JavaScript可以在此基础上构建其他功能。这引出了特性的第一个亮点：`using`声明！`using`是一个新关键字，允许我们声明新的固定绑定，有点像`const`。关键区别在于，用`using`声明的变量在作用域结束时会被调用其`Symbol.dispose`方法！所以我们可以简单地将代码写成这样：

In [ ]:
export function doSomeWork() {    using file = new TempFile(".some_temp_file");    // use file...    if (someCondition()) {        // do some more work...        return;    }}

看看这个——没有`try`/`finally`块！至少，我们看不到它们。从功能上讲，这正是`using`声明将为我们做的事情，但我们不必处理它。您可能熟悉C#中的[`using`声明](https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/proposals/csharp-8.0/using)、Python中的[`with`语句](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement)或Java中的[`try`-with-resource声明](https://docs.oracle.com/javase/tutorial/essential/exceptions/tryResourceClose.html)。这些都与JavaScript的新`using`关键字类似，并提供了在作用域结束时或像`return`或抛出的错误这样的“提前返回”之前执行对象“销毁”的类似显式方法。`using`声明在它们包含的作用域的最末端或在像`return`或抛出的错误这样的“提前返回”之前进行清理。它们还像堆栈一样按照后进先出的顺序进行处置。

In [ ]:
function loggy(id: string): Disposable {    console.log(`Creating ${id}`);    return {        [Symbol.dispose]() {            console.log(`Disposing ${id}`);        }    }}function func() {    using a = loggy("a");    using b = loggy("b");    {        using c = loggy("c");        using d = loggy("d");    }    using e = loggy("e");    return;    // Unreachable.    // Never created, never disposed.    using f = loggy("f");}func();// Creating a// Creating b// Creating c// Creating d// Disposing d// Disposing c// Creating e// Disposing e// Disposing b// Disposing a

`using` 声明应该对异常具有弹性；如果抛出错误，它会在处置后重新抛出。另一方面，函数的主体可能按预期执行，但 `Symbol.dispose` 可能会抛出异常。在这种情况下，该异常也会被重新抛出。但如果在处置之前和期间的逻辑都抛出错误会发生什么？对于这些情况，引入了 `SuppressedError` 作为 `Error` 的新子类型。它具有一个 `suppressed` 属性，用于保存最后抛出的错误，以及一个 `error` 属性，用于保存最近抛出的错误。

In [ ]:
class ErrorA extends Error {    name = "ErrorA";}class ErrorB extends Error {    name = "ErrorB";}function throwy(id: string) {    return {        [Symbol.dispose]() {            throw new ErrorA(`Error from ${id}`);        }    };}function func() {    using a = throwy("a");    throw new ErrorB("oops!")}try {    func();}catch (e: any) {    console.log(e.name); // SuppressedError    console.log(e.message); // An error was suppressed during disposal.    console.log(e.error.name); // ErrorA    console.log(e.error.message); // Error from a    console.log(e.suppressed.name); // ErrorB    console.log(e.suppressed.message); // oops!}

你可能已经注意到，在这些示例中我们使用了同步方法。然而，许多资源处理涉及到*异步*操作，我们需要等待这些操作完成，然后才能继续运行其他代码。这就是为什么还有一个新的`Symbol.asyncDispose`，它引出了节目的下一个明星——`await using`声明。这些与`using`声明类似，但关键是它们查找必须`await`的处置。它们使用一个名为`Symbol.asyncDispose`的不同方法，尽管它们也可以操作任何具有`Symbol.dispose`的对象。为了方便起见，TypeScript还引入了一个名为`AsyncDisposable`的全局类型，该类型描述任何具有异步处置方法的的对象。

In [ ]:
async function doWork() {    // Do fake work for half a second.    await new Promise(resolve => setTimeout(resolve, 500));}function loggy(id: string): AsyncDisposable {    console.log(`Constructing ${id}`);    return {        async [Symbol.asyncDispose]() {            console.log(`Disposing (async) ${id}`);            await doWork();        },    }}async function func() {    await using a = loggy("a");    await using b = loggy("b");    {        await using c = loggy("c");        await using d = loggy("d");    }    await using e = loggy("e");    return;    // Unreachable.    // Never created, never disposed.    await using f = loggy("f");}func();// Constructing a// Constructing b// Constructing c// Constructing d// Disposing (async) d// Disposing (async) c// Constructing e// Disposing (async) e// Disposing (async) b// Disposing (async) a

定义类型为`Disposable`和`AsyncDisposable`可以使您的代码更易于他人处理，如果您期望他们始终如一地执行销毁逻辑。实际上，在野外存在许多现有的类型，它们具有`dispose()`或`close()`方法。例如，Visual Studio Code API甚至定义了[它们自己的`Disposable`接口](https://code.visualstudio.com/api/references/vscode-api#Disposable)。浏览器中的API以及Node.js、Deno和Bun等运行时可能会选择使用`Symbol.dispose`和`Symbol.asyncDispose`，用于已经有清理方法的对象，如文件句柄、连接等。现在，也许这一切对于库来说听起来很棒，但对于您的场景来说可能有点过于繁重。如果您正在进行大量的临时清理，创建一个新类型可能会引入很多过度抽象和对最佳实践的问题。例如，再次以我们的`TempFile`示例为例。

In [ ]:
class TempFile implements Disposable {    #path: string;    #handle: number;    constructor(path: string) {        this.#path = path;        this.#handle = fs.openSync(path, "w+");    }    // other methods    [Symbol.dispose]() {        // Close the file and delete it.        fs.closeSync(this.#handle);        fs.unlinkSync(this.#path);    }}export function doSomeWork() {    using file = new TempFile(".some_temp_file");    // use file...    if (someCondition()) {        // do some more work...        return;    }}

我们所想要的只是记住调用两个函数——但这是编写它的最佳方式吗？我们应该在构造函数中调用`openSync`，创建一个`open()`方法，还是自己传递句柄？我们应该为需要执行的每个可能操作提供一个方法，还是只将属性设置为公共的？这引出了特性的最后两个明星：`DisposableStack`和`AsyncDisposableStack`。这些对象对于执行一次性清理以及任意数量的清理非常有用。`DisposableStack`是一个具有多个方法的对象，用于跟踪`Disposable`对象，并可以为其提供执行任意清理工作的函数。我们还可以将它们分配给`using`变量，因为——猜猜看——*它们也是`Disposable`*！所以，这就是我们原本可以编写示例的方式。

In [ ]:
function doSomeWork() {    const path = ".some_temp_file";    const file = fs.openSync(path, "w+");    using cleanup = new DisposableStack();    cleanup.defer(() => {        fs.closeSync(file);        fs.unlinkSync(path);    });    // use file...    if (someCondition()) {        // do some more work...        return;    }    // ...}

在这里，`defer()` 方法只接受一个回调函数，该回调函数将在 `cleanup` 被处理后运行。通常，`defer`（以及其他 `DisposableStack` 方法，如 `use` 和 `adopt`）应在创建资源后立即调用。顾名思义，`DisposableStack` 像栈一样按照先进后出的顺序处理它跟踪的所有内容，因此，在创建值后立即 `defer` 有助于避免奇怪的依赖问题。`AsyncDisposableStack` 的工作原理类似，但可以跟踪 `async` 函数和 `AsyncDisposable`，并且本身是一个 `AsyncDisposable`。`defer` 方法在许多方面与 [Go](https://go.dev/tour/flowcontrol/12)、[Swift](https://docs.swift.org/swift-book/documentation/the-swift-programming-language/statements/#Defer-Statement)、[Zig](https://ziglang.org/documentation/master/#defer)、[Odin](https://odin-lang.org/docs/overview/#defer-statement) 等语言中的 `defer` 关键字类似，其约定应该相似。由于这个特性非常新，大多数运行时都不会原生支持它。要使用它，您将需要为以下内容提供运行时 polyfills：* `Symbol.dispose`* `Symbol.asyncDispose`* `DisposableStack`* `AsyncDisposableStack`* `SuppressedError`然而，如果您只关心 `using` 和 `await using`，那么您应该只需要 polyfilling 内置的 `symbol`。像下面这样简单的东西应该适用于大多数情况：

In [ ]:
Symbol.dispose ??= Symbol("Symbol.dispose");Symbol.asyncDispose ??= Symbol("Symbol.asyncDispose");

您还需要将编译的`target`设置为`es2022`或更低版本，并将`lib`设置配置为包含`"esnext"`或`"esnext.disposable"`。```json{    "compilerOptions": {        "target": "es2022",        "lib": ["es2022", "esnext.disposable", "dom"]    }}```有关此功能的更多信息，请[查看GitHub上的工作](https://github.com/microsoft/TypeScript/pull/54505)！## 装饰器元数据TypeScript 5.2实现了[一个即将推出的ECMAScript特性，称为装饰器元数据](https://github.com/tc39/proposal-decorator-metadata)。这个特性的关键思想是使装饰器能够轻松地在它们所使用或所在的任何类上创建和使用元数据。现在，每当使用装饰器函数时，它们就可以访问其上下文对象上的一个新的`metadata`属性。`metadata`属性只包含一个简单的对象。由于JavaScript允许我们任意添加属性，它可以被用作一个字典，由每个装饰器更新。或者，由于每个`metadata`对象对于类的每个装饰部分都是相同的，它可以被用作`Map`中的一个键。在类上或类中的所有装饰器运行之后，可以通过`Symbol.metadata`在类上访问该对象。

In [ ]:
interface Context {    name: string;    metadata: Record<PropertyKey, unknown>;}function setMetadata(_target: any, context: Context) {    context.metadata[context.name] = true;}class SomeClass {    @setMetadata    foo = 123;    @setMetadata    accessor bar = "hello!";    @setMetadata    baz() { }}const ourMetadata = SomeClass[Symbol.metadata];console.log(JSON.stringify(ourMetadata));// { "bar": true, "baz": true, "foo": true }

这在许多不同的场景中都非常有用。元数据可能会附加用于许多用途，如调试、序列化或使用装饰器执行依赖注入。由于元数据对象是针对每个装饰类创建的，因此框架可以私下将它们用作`Map`或`WeakMap`的键，或者根据需要添加属性。例如，假设我们想使用装饰器来跟踪在使用`JSON.stringify`时哪些属性和访问器是可序列化的，如下所示：

In [ ]:
import { serialize, jsonify } from "./serializer";class Person {    firstName: string;    lastName: string;    @serialize    age: number    @serialize    get fullName() {        return `${this.firstName} ${this.lastName}`;    }    toJSON() {        return jsonify(this)    }    constructor(firstName: string, lastName: string, age: number) {        // ...    }}

在这里，意图是只有`age`和`fullName`应该被序列化，因为它们被标记了`@serialize`装饰器。我们为此定义了一个`toJSON`方法，但它只是调用了`jsonify`，而`jsonify`使用了`@serialize`创建的元数据。以下是模块`./serialize.ts`可能的定义示例：

In [ ]:
const serializables = Symbol();type Context =    | ClassAccessorDecoratorContext    | ClassGetterDecoratorContext    | ClassFieldDecoratorContext    ;export function serialize(_target: any, context: Context): void {    if (context.static || context.private) {        throw new Error("Can only serialize public instance members.")    }    if (typeof context.name === "symbol") {        throw new Error("Cannot serialize symbol-named properties.");    }    const propNames =        (context.metadata[serializables] as string[] | undefined) ??= [];    propNames.push(context.name);}export function jsonify(instance: object): string {    const metadata = instance.constructor[Symbol.metadata];    const propNames = metadata?.[serializables] as string[] | undefined;    if (!propNames) {        throw new Error("No members marked with @serialize.");    }    const pairStrings = propNames.map(key => {        const strKey = JSON.stringify(key);        const strValue = JSON.stringify((instance as any)[key]);        return `${strKey}: ${strValue}`;    });    return `{ ${pairStrings.join(", ")} }`;}

这个模块有一个名为`serializables`的本地`symbol`，用于存储和检索标记为`@serializable`的属性名称。每次调用`@serializable`时，它将这些属性名称的列表存储在元数据上。当调用`jsonify`时，从元数据中获取属性列表，并用于从实例中检索实际值，最终序列化这些名称和值。从技术上讲，使用`symbol`会使得这些数据对其他人是可访问的。另一种选择可能是使用一个`WeakMap`，将元数据对象作为键。这样可以保持数据的私密性，并且在这种情况下使用更少的类型断言，但在其他方面类似。

In [ ]:
const serializables = new WeakMap<object, string[]>();type Context =    | ClassAccessorDecoratorContext    | ClassGetterDecoratorContext    | ClassFieldDecoratorContext    ;export function serialize(_target: any, context: Context): void {    if (context.static || context.private) {        throw new Error("Can only serialize public instance members.")    }    if (typeof context.name !== "string") {        throw new Error("Can only serialize string properties.");    }    let propNames = serializables.get(context.metadata);    if (propNames === undefined) {        serializables.set(context.metadata, propNames = []);    }    propNames.push(context.name);}export function jsonify(instance: object): string {    const metadata = instance.constructor[Symbol.metadata];    const propNames = metadata && serializables.get(metadata);    if (!propNames) {        throw new Error("No members marked with @serialize.");    }    const pairStrings = propNames.map(key => {        const strKey = JSON.stringify(key);        const strValue = JSON.stringify((instance as any)[key]);        return `${strKey}: ${strValue}`;    });    return `{ ${pairStrings.join(", ")} }`;}

作为一个注释，这些实现并不处理子类和继承。这留给你作为一个练习（你可能会发现，在一个版本的文件中比另一个版本更容易！）。因为这个特性还很新，大多数运行时都不会原生支持它。要使用它，你需要一个 `Symbol.metadata` 的 polyfill。像下面这样简单的东西应该适用于大多数情况：

In [ ]:
Symbol.metadata ??= Symbol("Symbol.metadata");

您还需要将编译的`target`设置为`es2022`或更低版本，并配置您的`lib`设置以包含`"esnext"`或`"esnext.decorators"`。```json{    "compilerOptions": {        "target": "es2022",        "lib": ["es2022", "esnext.decorators", "dom"]    }}```我们要感谢[Oleksandr Tarasiuk](https://github.com/a-tarasyuk)为TypeScript 5.2贡献了[装饰器元数据的实现](https://github.com/microsoft/TypeScript/pull/54657)！<!-- TODO: 为什么在`Symbol.metadata`的存在周围有一个条件类型？ -->## 命名和匿名元组元素元组类型已经支持每个元素的可选标签或名称。

In [ ]:
type Pair<T> = [first: T, second: T];

这些标签不会改变你可以用它们做什么——它们仅仅是为了帮助可读性和工具支持。然而，TypeScript 之前有一个规则，即元组中不能混合使用带标签和不带标签的元素。换句话说，要么元组中没有元素可以有标签，要么所有元素都需要一个标签。

In [ ]:
// ✅ fine - no labelstype Pair1<T> = [T, T];// ✅ fine - all fully labeledtype Pair2<T> = [first: T, second: T];// ❌ previously an errortype Pair3<T> = [first: T, T];//                         ~// Tuple members must all have names// or all not have names.

这可能会让剩余元素变得烦人，因为我们不得不添加一个类似 `rest` 或 `tail` 的标签。

In [ ]:
// ❌ previously an errortype TwoOrMore_A<T> = [first: T, second: T, ...T[]];//                                          ~~~~~~// Tuple members must all have names// or all not have names.// ✅type TwoOrMore_B<T> = [first: T, second: T, rest: ...T[]];

这也意味着这种限制必须在类型系统内部强制执行，意味着TypeScript将丢失标签。

In [ ]:
type HasLabels = [a: string, b: string];type HasNoLabels = [number, number];type Merged = [...HasNoLabels, ...HasLabels];//   ^ [number, number, string, string]////     'a' and 'b' were lost in 'Merged'

在 TypeScript 5.2 中，对元组标签的全有或全无限制已经解除。现在，在将标签展开到未标记的元组时，语言还可以保留标签。我们要感谢 [Josh Goldberg](https://github.com/JoshuaKGoldberg) 和 [Mateusz Burzyński](https://github.com/Andarist)，他们[合作解除了这个限制](https://github.com/microsoft/TypeScript/pull/53356)。## 更简单的数组联合方法使用在之前的 TypeScript 版本中，对数组联合调用方法可能会导致痛苦。

In [ ]:
declare let array: string[] | number[];array.filter(x => !!x);//    ~~~~~~ error!// This expression is not callable.//   Each member of the union type '...' has signatures,//   but none of those signatures are compatible//   with each other.

在这个例子中，TypeScript会尝试看看每个版本的`filter`是否在`string[]`和`number[]`之间兼容。如果没有一个连贯的策略，TypeScript会双手一摊说“我做不到”。在TypeScript 5.2中，在这些情况下放弃之前，数组的联合被视为一个特例。一个新的数组类型是由每个成员元素类型构成的，然后在该类型上调用方法。以上面的例子为例，`string[] | number[]`被转换成`(string | number)[]`（或`Array<string | number>`），然后在那个类型上调用`filter`。这里有一个小小的注意事项，那就是`filter`将产生一个`Array<string | number>`，而不是`string[] | number[]`；但对于一个新产生的值来说，出问题的风险较小。这意味着像`filter`、`find`、`some`、`every`和`reduce`这样的许多方法都应该在数组联合上可调用，而在以前这是不可能的。你可以[阅读更多关于实现拉取请求的细节](https://github.com/microsoft/TypeScript/pull/53489)。## 带有TypeScript实现文件扩展名的仅类型导入路径TypeScript现在允许在仅类型导入路径中包含声明*和*实现文件扩展名，无论是否启用了`allowImportingTsExtensions`。这意味着你现在可以编写使用`.ts`、`.mts`、`.cts`和`.tsx`文件扩展名的`import type`语句。

In [ ]:
import type { JustAType } from "./justTypes.ts";export function f(param: JustAType) {    // ...}

这也意味着可以在 TypeScript 和 JavaScript 中使用 JSDoc 的 `import()` 类型，可以使用这些文件扩展名。

In [ ]:
/** * @param {import("./justTypes.ts").JustAType} param */export function f(param) {    // ...}

更多信息，请[参见此处的更改](https://github.com/microsoft/TypeScript/pull/54746)。## 对象成员的逗号补全在向对象添加新属性时，很容易忘记添加逗号。以前，如果你忘记逗号并请求自动补全，TypeScript会令人困惑地给出不相关的补全结果。TypeScript 5.2 现在在缺少逗号时优雅地提供对象成员补全。但为了避免让你遇到语法错误，它还会自动插入缺失的逗号。![尽管在先前属性后缺少逗号，对象字面量中的属性仍被补全。当属性名称被补全时，缺失的逗号会自动插入。](https://devblogs.microsoft.com/typescript/wp-content/uploads/sites/11/2023/06/comma-completions-5-2-beta.gif)更多信息，请[参见此处的实现](https://github.com/microsoft/TypeScript/pull/52899)。## 内联变量重构TypeScript 5.2 现在有一个重构功能，可以将变量的内容内联到所有使用位置。![一个名为 'path' 的变量初始化为字符串，其两个使用位置都被替换](https://devblogs.microsoft.com/typescript/wp-content/uploads/sites/11/2023/06/inline-variable-5-2-beta.gif)。使用“内联变量”重构将消除该变量，并将其所有使用位置替换为其初始化表达式。请注意，这可能会导致初始化表达式的副作用在不同的时间运行，以及根据变量的使用次数运行多次。更多细节，请[参见实现拉取请求](https://github.com/microsoft/TypeScript/pull/54281)。<!-- Inlay Parameter Hints -->## 优化进行中的类型兼容性检查由于 TypeScript 是一个结构化类型系统，有时需要以成员方式比较类型；然而，递归类型在这里增加了一些问题。例如：

In [ ]:
interface A {    value: A;    other: string;}interface B {    value: B;    other: number;}

当检查类型`A`是否与类型`B`兼容时，TypeScript将最终检查`A`和`B`中的`value`类型是否分别兼容。此时，类型系统需要停止进一步检查，并继续检查其他成员。为此，类型系统必须跟踪何时任何两个类型已经在相关联。以前，TypeScript已经保留了一个类型对栈，并通过遍历该栈来确定这些类型是否正在相关联。当这个栈不深时，这不是问题；但当栈不浅时，呃，[这是个问题](https://accidentallyquadratic.tumblr.com/)。在TypeScript 5.3中，一个简单的`Set`有助于跟踪这些信息。这使使用[drizzle](https://github.com/drizzle-team/drizzle-orm)库的已报告测试用例的时间减少了33%以上！```基准测试1：旧版  时间（平均值±σ）：3.115秒±0.067秒 [用户：4.403秒，系统：0.124秒]  范围（最小…最大）：3.018秒…3.196秒 10次运行 基准测试2：新版  时间（平均值±σ）：2.072秒±0.050秒 [用户：3.355秒，系统：0.135秒]  范围（最小…最大）：1.985秒…2.150秒 10次运行 总结  '新版'运行速度比'旧版'快    1.50 ± 0.05倍```[在此处阅读有关更改的更多信息](https://github.com/microsoft/TypeScript/pull/55224)。## 破坏性更改和正确性修复TypeScript努力不引入不必要的破坏；然而，我们偶尔必须进行更正和改进，以便更好地分析代码。### `lib.d.ts`更改为DOM生成的类型可能会影响您的代码库。有关更多信息，请[参阅TypeScript 5.2的DOM更新](https://github.com/microsoft/TypeScript/pull/54725)。### `labeledElementDeclarations`可能包含`undefined`元素为了[支持标记和未标记元素的混合](https://github.com/microsoft/TypeScript/pull/53356)，TypeScript的API略有变化。`TupleType`的`labeledElementDeclarations`属性可能在每个未标记元素的位置包含`undefined`。```diff  interface TupleType {-     labeledElementDeclarations?: readonly (NamedTupleMember | ParameterDeclaration)[];+     labeledElementDeclarations?: readonly (NamedTupleMember | ParameterDeclaration | undefined)[];  }```### 在最近的Node.js设置下，`module`和`moduleResolution`必须匹配`--module`和`--moduleResolution`选项都支持`node16`和`nodenext`设置。这些实际上是“现代Node.js”设置，应该在任何最近的Node.js项目上使用。我们发现，当这两个选项在是否使用Node.js相关设置上不一致时，项目实际上是配置错误的。在TypeScript 5.2中，当使用`node16`或`nodenext`作为`--module`和`--moduleResolution`选项中的任何一个时，TypeScript现在要求另一个具有类似的Node.js相关设置。在设置不一致的情况下，您可能会收到类似以下内容的错误消息：```选项'moduleResolution'必须设置为'NodeNext'（或保持未指定），当选项'module'设置为'NodeNext'时。```或者```选项'module'必须设置为'Node16'，当选项'moduleResolution'设置为'Node16'时。```例如，`--module esnext --moduleResolution node16`将被拒绝 - 但您可能最好只使用`--module nodenext`，或者`--module esnext --moduleResolution bundler`。有关更多信息，请[参阅此处的更改](https://github.com/microsoft/TypeScript/pull/54567)。### 对合并符号的一致导出检查当两个声明合并时，它们必须同意它们是否都导出。由于一个错误，TypeScript错过了环境上下文中的特定情况，如声明文件或`declare module`块。例如，它不会对以下情况发出错误，其中`replaceInFile`一次声明为导出的函数，一次声明为未导出的命名空间。

In [ ]:
declare module 'replace-in-file' {    export function replaceInFile(config: unknown): Promise<unknown[]>;    export {};    namespace replaceInFile {        export function sync(config: unknown): unknown[];  }}

在一个环境模块中，添加一个 `export { ... }` 或类似的构造，如 `export default ...`，会隐式地改变所有声明是否自动导出的行为。TypeScript 现在更加一致地识别这些令人困惑的语义，并在 `replaceInFile` 的所有声明需要在其修饰符上达成一致的事实上发出错误，并将发出以下错误：```合并声明 'replaceInFile' 中的个别声明必须全部导出或全部本地。```有关更多信息，请[查看此处的更改](https://github.com/microsoft/TypeScript/pull/54659)。